This notebook aims to use pycaret on the CICDDoS2019 dataset split according to the [original release paper](https://ieeexplore.ieee.org/abstract/document/8888419).

we show the proportions of Anomaly/Total data points caluclated from inputting the following into the excel sheet\
`=COUNTIF(CK:CK,"Syn")/(COUNTIF(CK:CK,"Syn")+COUNTIF(CK:CK,"BENIGN"))`
loading the large csv into the data frame took too much time so this faster method was used.

01-12 folder as the training data
```
['DrDoS_SSDP.csv': 99.971%
'DrDoS_NTP.csv': 98.7062%
'TFTP.csv', 
'UDPLag.csv' : 98.9991% of 370166 rows
'DrDoS_UDP.csv', 99.8919% of 1048575 rows
'Syn.csv' : 99.9966%
'DrDoS_MSSQL.csv', 
'DrDoS_SNMP.csv', 
'DrDoS_DNS.csv', 
'DrDoS_LDAP.csv':99.9256%]
```

03-12 as the testing data
```
['LDAP.csv':
 'MSSQL.csv':
 'NetBIOS.csv':
 'Portmap.csv': 97.5304% of 191694
 'Syn.csv':
 'UDP.csv':
 'UDPLag.csv': ]
```

and attempts to use all categories

In [1]:
print("we can assume we have enough BENIGN in each CSV to sample to make the proportion clores to 50%. for example, this is how many BENIGN rows we have in UDPLag.csv: ", 370166* (100-98.9991) / 100)

we can assume we have enough BENIGN in each CSV to sample to make the proportion clores to 50%. for example, this is how many BENIGN rows we have in UDPLag.csv:  3704.9914940000053


luckally, pycaret has tons of features to address these data issues written below

In [2]:
import dask.dataframe as dd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pycaret
import re
%matplotlib inline 

listing columns names

In [3]:
columns = ['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio',
       ' Average Packet Size', ' Avg Fwd Segment Size',
       ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
       ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes',
       'Init_Win_bytes_forward', ' Init_Win_bytes_backward',
       ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean',
       ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std',
       ' Idle Max', ' Idle Min', 'SimillarHTTP', ' Inbound', ' Label']
ignore_columns = ['Unnamed: 0', 'Flow Bytes/s', ' Flow Packets/s', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port',]
usecols = [x for x in columns if x not in ignore_columns]

strictly type to minimize memory usage. defined via previously made function

In [4]:
dtype = {
    'Unnamed: 0': 'int64',
    'Flow ID': 'object',
    ' Source IP': 'object',
    ' Source Port': 'int64',
    ' Destination IP': 'object',
    ' Destination Port': 'int64',
    ' Protocol': 'int64',
    ' Timestamp': 'object',
    ' Flow Duration': 'int64',
    ' Total Fwd Packets': 'int64',
    ' Total Backward Packets': 'int64',
    'Total Length of Fwd Packets': 'float64',
    ' Total Length of Bwd Packets': 'float64',
    ' Fwd Packet Length Max': 'float64',
    ' Fwd Packet Length Min': 'float64',
    ' Fwd Packet Length Mean': 'float64',
    ' Fwd Packet Length Std': 'float64',
    'Bwd Packet Length Max': 'float64',
    ' Bwd Packet Length Min': 'float64',
    ' Bwd Packet Length Mean': 'float64',
    ' Bwd Packet Length Std': 'float64',
    'Flow Bytes/s': 'float64',
    ' Flow Packets/s': 'float64',
    ' Flow IAT Mean': 'float64',
    ' Flow IAT Std': 'float64',
    ' Flow IAT Max': 'float64',
    ' Flow IAT Min': 'float64',
    'Fwd IAT Total': 'float64',
    ' Fwd IAT Mean': 'float64',
    ' Fwd IAT Std': 'float64',
    ' Fwd IAT Max': 'float64',
    ' Fwd IAT Min': 'float64',
    'Bwd IAT Total': 'float64',
    ' Bwd IAT Mean': 'float64',
    ' Bwd IAT Std': 'float64',
    ' Bwd IAT Max': 'float64',
    ' Bwd IAT Min': 'float64',
    'Fwd PSH Flags': 'int64',
    ' Bwd PSH Flags': 'int64',
    ' Fwd URG Flags': 'int64',
    ' Bwd URG Flags': 'int64',
    ' Fwd Header Length': 'int64',
    ' Bwd Header Length': 'int64',
    'Fwd Packets/s': 'float64',
    ' Bwd Packets/s': 'float64',
    ' Min Packet Length': 'float64',
    ' Max Packet Length': 'float64',
    ' Packet Length Mean': 'float64',
    ' Packet Length Std': 'float64',
    ' Packet Length Variance': 'float64',
    'FIN Flag Count': 'int64',
    ' SYN Flag Count': 'int64',
    ' RST Flag Count': 'int64',
    ' PSH Flag Count': 'int64',
    ' ACK Flag Count': 'int64',
    ' URG Flag Count': 'int64',
    ' CWE Flag Count': 'int64',
    ' ECE Flag Count': 'int64',
    ' Down/Up Ratio': 'float64',
    ' Average Packet Size': 'float64',
    ' Avg Fwd Segment Size': 'float64',
    ' Avg Bwd Segment Size': 'float64',
    ' Fwd Header Length.1': 'int64',
    'Fwd Avg Bytes/Bulk': 'int64',
    ' Fwd Avg Packets/Bulk': 'int64',
    ' Fwd Avg Bulk Rate': 'int64',
    ' Bwd Avg Bytes/Bulk': 'int64',
    ' Bwd Avg Packets/Bulk': 'int64',
    'Bwd Avg Bulk Rate': 'int64',
    'Subflow Fwd Packets': 'int64',
    ' Subflow Fwd Bytes': 'int64',
    ' Subflow Bwd Packets': 'int64',
    ' Subflow Bwd Bytes': 'int64',
    'Init_Win_bytes_forward': 'int64',
    ' Init_Win_bytes_backward': 'int64',
    'act_data_pkt_fwd': 'int64',
    'min_seg_size_forward': 'int64',
    'Active Mean': 'float64',
    'Active Std': 'float64',
    'Active Max': 'float64',
    'Active Min': 'float64',
    'Idle Mean': 'float64',
    'Idle Std': 'float64',
    'Idle Max': 'float64',
    'Idle Min': 'float64',
    'SimillarHTTP': 'object',
    'Inbound': 'int64',
    'Label': 'object'
}

Setting up [pycaret](https://pycaret.readthedocs.io/en/latest/api/classification.html#pycaret.classification.setup) to test 5 models.\
we are using the following features for the following reasons:\
`fix_imbalance` balances the distributions\
`transformation` makes the distribution gaussian\
`normalize` normalizes features\
`pca` reduces the features we use\
`pca_method` is `incremental` for we are using a large dataset\
`feature_selection` selects few features to use\
`n_features_to_select` specifies how many or fractions of features to start with\
`numeric_imputation` is done via knn to accurately impute\
`imputation_type` is `iterative` to be accurate\
`polynomial_features` generates features. can adjust n variables allowed with\
`log_experiment` to view the experiments\
`use_gpu` to speed up algorithms\

If we want to use algorithms that can't handle categories, set "max_encoding_ohe=-1" 

In [7]:
from pycaret.classification import *
#with MLflow
#s = setup(test_data=test_df, log_plots=True, log_profile=True, log_data=True, profile=True, use_gpu=True, data=train_df, target='_Label', log_experiment=True, feature_selection=True, n_features_to_select=0.5, numeric_imputation='knn',  imputation_type='iterative', polynomial_features=True, remove_multicollinearity=True, fix_imbalance=True, normalize=True, transformation=True, pca=True, pca_method='incremental')

#without MLflow
# fugue tutorials reccomends sending data loading function
# Kentaro's PC has 14 cores
syn_merged_dir = 'C:\\Users\\ktv07101\\Desktop\\BHNI Anomaly Detection Related\\CondaModelReplication\\CICDDoS2019\\CSV-03-11\\03-11\\Syn.csv'
s = setup(
          session_id=1, 
          n_jobs=14, 
          profile=True, 
          use_gpu=True, 
          data_func= lambda: pd.read_csv(syn_merged_dir, usecols=usecols, dtype=dtype),
          target=" Label", 
          feature_selection=True, 
          n_features_to_select=0.5, 
          numeric_imputation='knn',  
          imputation_type='iterative', 
          # polynomial_features=True, This causes 74GB RAM to be allocated
          remove_multicollinearity=True, 
          fix_imbalance=True, 
          normalize=True, 
          transformation=True, 
          pca=True, 
          pca_method='incremental')

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are n

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves t

[LightGBM] [Info] Number of positive: 2999325, number of negative: 2999325
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.334525 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11730
[LightGBM] [Info] Number of data points in the train set: 5998650, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


,Description,Value
0,Session id,1
1,Target,Label
2,Target type,Binary
3,Target mapping,"BENIGN: 0, Syn: 1"
4,Original data shape,"(4320541, 81)"
5,Transformed data shape,"(7294813, 41)"
6,Transformed train set shape,"(5998650, 41)"
7,Transformed test set shape,"(1296163, 41)"
8,Numeric features,77
9,Categorical features,3


Loading profile... Please Wait!
Profiler Failed. No output to show, continue with modeling.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the

In [ ]:
from pycaret.parallel import FugueBackend
models = compare_models(fold = 10,  
                        round = 4,  
                        sort = "Accuracy", 
                        turbo = True, 
                        n_select=5, 
                        #parallel=FugueBackend("dask"), CancelError prompt python access to device, accepting doesn't change this
                        verbose=False)

In [ ]:
results = pull().reset_index(drop=True)
results.iloc[0:5]